In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=5aeb12fd1e467c36ad49e5c571e7eba13e9cff60375e6e9b233d3ba967384334
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
!pip install transformers

     |████████████████████████████████| 542kB 6.7MB/s 
     |████████████████████████████████| 3.7MB 30.6MB/s 
     |████████████████████████████████| 1.0MB 39.2MB/s 
     |████████████████████████████████| 870kB 45.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=d791a6186b6365085b664f3da2d9ca890182c2d7da8cc0d1100e9385034ee33e
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [3]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [4]:
import torch
if torch.cuda.is_available():     # If there's a GPU available...        
    device = torch.device("cuda")   # Tell PyTorch to use the GPU.
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [0]:
import tensorflow as tf

In [6]:
import wget
import os

print('Downloading dataset...')
url = 'http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv'
 
if not os.path.exists('./qqp2.tsv'):
    wget.download(url, './qqp2.tsv')

In [7]:
os.listdir()

['.config', 'qqp2.tsv', 'sample_data']

In [8]:
import pandas as pd
file = pd.read_csv('qqp2.tsv', delimiter='\t')
file.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
q1 = file['question1'].values
q2 = file['question2'].values
labels = file['is_duplicate'].values

In [10]:
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [12]:
print(' Original: ', q1[10], q2[10])

print('Tokenized: ', tokenizer.tokenize(q1[10]), tokenizer.tokenize(q2[10]))

print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(q1[10])), tokenizer.convert_tokens_to_ids(tokenizer.tokenize(q2[10])))

 Original:  Method to find separation of slits using fresnel biprism? What are some of the things technicians can tell about the durability and reliability of Laptops and its components?
Tokenized:  ['method', 'to', 'find', 'separation', 'of', 'slits', 'using', 'fr', '##es', '##nel', 'bi', '##pr', '##ism', '?'] ['what', 'are', 'some', 'of', 'the', 'things', 'technicians', 'can', 'tell', 'about', 'the', 'du', '##ra', '##bility', 'and', 'reliability', 'of', 'laptop', '##s', 'and', 'its', 'components', '?']
Token IDs:  [4118, 2000, 2424, 8745, 1997, 29199, 2478, 10424, 2229, 11877, 12170, 18098, 2964, 1029] [2054, 2024, 2070, 1997, 1996, 2477, 20202, 2064, 2425, 2055, 1996, 4241, 2527, 8553, 1998, 15258, 1997, 12191, 2015, 1998, 2049, 6177, 1029]


In [13]:
print(len(q1))
print(len(q2))

404290
404290


In [0]:
input_ids = []
lab = []
for i in range(len(q1)):
    if type(q1[i])==str and type(q2[i])==str:
        encoded_pair = tokenizer.encode(q1[i], q2[i], add_special_tokens=True)
        input_ids.append(encoded_pair)
        lab.append(labels[i])

In [0]:
len(input_ids)

404287

In [0]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  330


In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 384

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 384 values...

Padding token: "[PAD]", ID: 0

Done.


In [0]:
attention_masks = []
for pair in input_ids:
    att_mask = [int(token_id > 0) for token_id in pair]    
    attention_masks.append(att_mask)

In [0]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, lab, 
                                                            random_state=2018, test_size=0.3)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, lab,
                                             random_state=2018, test_size=0.3)

In [0]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
from transformers import BertForSequenceClassification, BertConfig 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                                                          num_labels = 2, 
                                                                                           output_attentions = False, 
                                                                                           output_hidden_states = False, 
                                                                                        )
model.cuda() 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 2e-5,  eps = 1e-8 )

In [0]:
import numpy as np
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed): #Takes a time in seconds and returns a string hh:mm:ss
    elapsed_rounded = int(round((elapsed)))    
    return str(datetime.timedelta(seconds=elapsed_rounded)) # Format as hh:mm:ss

In [0]:
import random

epochs = 10
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []
for epoch_i in range(epochs):
    t0 = time.time()
    total_loss = 0

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    model.train() # set model in train mode
    for step, batch in enumerate(train_dataloader):

        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)    
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...
  Batch   100  of  17,688.    Elapsed: 0:01:55.
  Batch   200  of  17,688.    Elapsed: 0:03:55.
  Batch   300  of  17,688.    Elapsed: 0:05:54.
  Batch   400  of  17,688.    Elapsed: 0:07:54.
  Batch   500  of  17,688.    Elapsed: 0:09:54.
  Batch   600  of  17,688.    Elapsed: 0:11:54.
  Batch   700  of  17,688.    Elapsed: 0:13:54.
  Batch   800  of  17,688.    Elapsed: 0:15:54.
  Batch   900  of  17,688.    Elapsed: 0:17:53.
  Batch 1,000  of  17,688.    Elapsed: 0:19:53.
  Batch 1,100  of  17,688.    Elapsed: 0:21:53.
  Batch 1,200  of  17,688.    Elapsed: 0:23:53.
  Batch 1,300  of  17,688.    Elapsed: 0:25:52.
  Batch 1,400  of  17,688.    Elapsed: 0:27:52.
  Batch 1,500  of  17,688.    Elapsed: 0:29:52.
  Batch 1,600  of  17,688.    Elapsed: 0:31:51.
  Batch 1,700  of  17,688.    Elapsed: 0:33:51.
  Batch 1,800  of  17,688.    Elapsed: 0:35:51.
  Batch 1,900  of  17,688.    Elapsed: 0:37:51.
  Batch 2,000  of  17,688.    Elapsed: 0:39: